In [1]:
from pprint import pprint
import json
import os

import pandas as pd

In [2]:
fpath = '../model_tuning/CNN_Nbands_expanded_balanced/'

In [21]:
trial_dict = {}
trial_count = 0
for item in os.listdir(fpath):
    path = os.path.join(fpath, item)
    if os.path.isdir(path):
        f = open(os.path.join(path, 'trial.json'))
        data = json.load(f)
        
        # Track existing keys to add any None values
        existing_keys = set(trial_dict.keys())
                
        temp_dict = data['hyperparameters']['values']
        
        print(temp_dict)
        
        macro_f1 = data['metrics']['metrics'].get('val_macro_f1', 0)
        accuracy = data['metrics']['metrics'].get('val_accuracy', 0)
        if macro_f1:
            macro_f1 = macro_f1['observations'][0]['value'][0]
        if accuracy:
            accuracy = accuracy['observations'][0]['value'][0]
        temp_dict['macro_f1'] = macro_f1
        temp_dict['accuracy'] = accuracy

        for k,v in temp_dict.items():
            if not trial_dict.get(k):
                trial_dict[k] = []
                
                # Append all None for previous trials
                for i in range(trial_count):
                    trial_dict[k].append(None)
                    
            trial_dict[k].append(v)
            
            if k in existing_keys:
                existing_keys.remove(k)
            
        if existing_keys:
            for k in existing_keys:
                trial_dict[k].append(None)
                
        # Update counter
        trial_count += 1

print(trial_dict)


{'activation': 'tanh', 'lr': 0.0003, 'layers': 1, 'nodes_1': 56, 'dropout_rate_1': 0.30000000000000004, 'batch_size': 16}
{'activation': 'relu', 'lr': 0.006, 'layers': 2, 'nodes_1': 24, 'dropout_rate_1': 0.1, 'batch_size': 120, 'nodes_2': 8, 'dropout_rate_2': 0.0}
{'activation': 'tanh', 'lr': 0.0001, 'layers': 1, 'nodes_1': 64, 'dropout_rate_1': 0.5, 'batch_size': 16, 'nodes_2': 16, 'dropout_rate_2': 0.4}
{'activation': 'tanh', 'lr': 0.0001, 'layers': 2, 'nodes_1': 64, 'dropout_rate_1': 0.5, 'batch_size': 16, 'nodes_2': 24, 'dropout_rate_2': 0.5}
{'activation': 'tanh', 'lr': 0.0001, 'layers': 1, 'nodes_1': 64, 'dropout_rate_1': 0.5, 'batch_size': 16, 'nodes_2': 8, 'dropout_rate_2': 0.5}
{'activation': 'tanh', 'lr': 0.0001, 'layers': 1, 'nodes_1': 64, 'dropout_rate_1': 0.5, 'batch_size': 16, 'nodes_2': 24, 'dropout_rate_2': 0.1}
{'activation': 'tanh', 'lr': 0.0001, 'layers': 1, 'nodes_1': 40, 'dropout_rate_1': 0.5, 'batch_size': 16, 'nodes_2': 16, 'dropout_rate_2': 0.35000000000000003}


In [26]:
trial_df = pd.DataFrame(trial_dict)
        
cols = trial_df.columns.tolist()
metrics = ['macro_f1', 'accuracy']
for m in metrics:
    cols.remove(m)

cols = cols + metrics
trial_df = trial_df[cols]
            
trial_df.sort_values(by='macro_f1').tail(1)

,activation,lr,layers,nodes_1,dropout_rate_1,batch_size,nodes_2,dropout_rate_2,nodes_3,dropout_rate_3,macro_f1,accuracy
11,relu,1.000e-04,3,64,0.0,16,16.0,0.0,8.0,0.0,0.927,0.914


In [26]:
trial_df = trial_df[trial_df['accuracy'] != 0]

In [33]:
trial_df.columns = ['hidden nodes', 'activation', 'dropout', 'learning rate', 'batch size', 'macro F1', 'Accuracy']

In [43]:
print_df = trial_df.sort_values(by='macro F1', ascending=True).round(decimals=4).reset_index(drop=True)
print_df.index += 1
print_df

,hidden nodes,activation,dropout,learning rate,batch size,macro F1,Accuracy
1,32,relu,0.50,0.0069,16,0.1310,0.2114
2,16,relu,0.20,0.0001,16,0.6283,0.7986
3,64,relu,0.50,0.0001,120,0.6920,0.7769
4,16,relu,0.00,0.0022,16,0.7366,0.8269
5,64,tanh,0.50,0.0001,16,0.7381,0.8106
6,64,relu,0.50,0.0001,16,0.8092,0.8473
7,56,relu,0.50,0.0003,32,0.8099,0.8426
8,64,tanh,0.00,0.0001,16,0.8321,0.8763
9,40,relu,0.20,0.0005,16,0.8447,0.8700
10,64,relu,0.00,0.0001,56,0.8573,0.8905


In [19]:
# Set precision level for metrics
pd.set_option('display.precision', 3)
cls_report = pd.read_csv('../models/CNN_Nbands_model_1657535394/classification_report.csv', index_col=0)
cls_report['support'] = cls_report['support'].astype(int)
cls_report['support']['accuracy'] = cls_report['support']['macro avg']
cls_report.rename(columns={'Unnamed: 0':'class'})
# cls_report.round(decimals=3)

/tmp/ipykernel_7376/1440962787.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cls_report['support']['accuracy'] = cls_report['support']['macro avg']


,precision,recall,f1-score,support
water,0.989,1.000,0.995,275
vegetation_trees,0.895,0.895,0.895,2801
vegetation_grass,0.805,0.828,0.816,1532
turf,0.990,0.960,0.975,860
impervious,0.897,0.942,0.919,1103
soil,0.975,0.947,0.960,2655
accuracy,0.914,0.914,0.914,9226
macro avg,0.925,0.929,0.927,9226
weighted avg,0.915,0.914,0.914,9226
